<a href="https://colab.research.google.com/github/dineshuthayakumar/MitraAIBootCamp/blob/main/Chatbot_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modern AI Pro: Simple Chatbot on your enterprise content

We will scrape the web content as well as our PDFs to answer user queries more effectively.


In [ ]:
!pip install --upgrade --quiet  sentence-transformers gradio bs4 pypdf2 chromadb rapidocr-onnxruntime langchain langchain-community langchainhub langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 869.7 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.6 MB/s 

## Step 1: Scraping our web content and using that as the foundation.


In [ ]:
# Load, chunk and index the contents of a blog/website.
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader = WebBaseLoader(web_path=("https://community.kissflow.com/category/documentation-section"))
docs = loader.load()
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
docs[0]

Document(metadata={'source': 'https://community.kissflow.com/category/documentation-section', 'title': 'Help Documentation - Kissflow Community', 'description': 'Read Kissflow help articles, how-to guides, and FAQs.', 'language': 'en'}, page_content="\n\n\nHelp Documentation - Kissflow Community\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n Community\n\n\n\n\nDeveloper Documentation Status\n\n\n\nHow to enable Javascript \n\n\n\n\n\n\nSign Up\n\nLogin\n\n\n\n\n\n\n\n\n\n\n\n     Welcome to the Kissflow Community!\n    \n\n\n\n\n\n\n\n\n\n\n     Search for answers here or ask a question\n\n\n\n\n\n\n\n\n\nProduct Documentation \n\nðŸŽ¬ Getting started\nðŸ“’ User guide\nðŸ”Ž Quick references\nðŸ’¼ Troubleshooting guide\n\nProduct Updates \n\nPre-release notes\nProduct announcements\nTips and tricks\n\nLatest Activity\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\nProduct Documentation\n\n\n\n\n\n\n\n\n\n\n\n\n\nProduct Documentation\n\n           Product Documentation\n      

In [ ]:
# Split and manage the texts
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [ ]:
# Store embeddings in a Vector DB
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cpu"})
vectorstore_web = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever_web = vectorstore_web.as_retriever()

<ipython-input-5-1d2d4e9d287b>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cpu"})
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optio

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from google.colab import userdata
import os

# If you haven't already installed "openai" in your Colab environment, make sure to:
# !pip install openai

# From LangChain, import the OpenAI LLM
from langchain.chat_models import ChatOpenAI

# Replace this with your actual environment variable (if stored in colab form fields, etc.)
os.environ["OPENAI_API_KEY"] = userdata.get("OpenAPIKey")

# Instantiate the OpenAI LLM (adjust model name if desired, e.g. "gpt-4" or "gpt-3.5-turbo")
llm = ChatOpenAI(model_name="gpt-4")

# Pull in the prompt from your prompt hub
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

# From your LangChain Core imports
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create your pipeline using OpenAI as the LLM
rag_web = (
    {"context": retriever_web | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
rag_web.invoke("Can I customise sub items of board?")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'The context does not provide specific information on whether you can customise sub items of a board on the Kissflow platform.'

In [ ]:
!pip install -q -U langchain-groq
from google.colab import userdata
import os
from langchain_groq import ChatGroq
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
llm = ChatGroq(model_name="llama3-70b-8192")

In [ ]:
# Retrieve and generate using the relevant snippets of the blog.
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
# Building the RAG
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
rag_web = (
    {"context": retriever_web | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_web.invoke("What do the users say about Modern AI Pro?")

"Users say that Modern AI Pro's training bootcamp is informative, eye-opening, and provides invaluable knowledge and skills. They express gratitude and appreciation for the training, and are excited to apply what they learned in real-world applications. Overall, the feedback is positive and recommends the bootcamp."

## Step 2: Extracting content from PDFs
Get one of our [research papers](https://drive.google.com/file/d/1kfjF9iuGG74ORFGu4v9dMO15pgsKI5Rh/view?usp=sharing) for sample. Download a copy locally and upload to the runtime.

In [ ]:
!gdown 1I8Ga_BK23OA8EEv3DtAmo5zw_Quud0ew

Downloading...
From: https://drive.google.com/uc?id=1I8Ga_BK23OA8EEv3DtAmo5zw_Quud0ew
To: /content/arso.pdf
100% 2.90M/2.90M [00:00<00:00, 101MB/s]


In [ ]:
from PyPDF2 import PdfReader
reader = PdfReader('arso.pdf')
text = ""
for i in range(0, len(reader.pages)):
    page = reader.pages[i]
    text += page.extract_text() + " "

In [ ]:
from langchain.schema.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = [Document(page_content=text, metadata={"source": "local"})]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=100)
all_splits = text_splitter.split_documents(documents)

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2", model_kwargs={"device": "cpu"})

vectordb_paper = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db_paper")
retriever_paper = vectordb_paper.as_retriever()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
all_splits

[Document(metadata={'source': 'local'}, page_content='Building Care Robots that Seniors Actually Care About: Results on\nLikability from a Survey of American Seniors\nBalaji Viswanathan1, Mahalakshmi Radhakrushnun1, Arvind Nagaraj1and Tim Oates2'),
 Document(metadata={'source': 'local'}, page_content='Balaji Viswanathan1, Mahalakshmi Radhakrushnun1, Arvind Nagaraj1and Tim Oates2\nAbstract — Over the past 2 decades, Care robots (CR) have'),
 Document(metadata={'source': 'local'}, page_content='Abstract — Over the past 2 decades, Care robots (CR) have\nbeen increasingly used to help seniors with a variety of tasks.\nThe COVID-19 pandemic accelerated the use of CR causing'),
 Document(metadata={'source': 'local'}, page_content='The COVID-19 pandemic accelerated the use of CR causing\nmixed reactions among seniors and caregivers. It is important\nto understand how to make robots that seniors actually care'),
 Document(metadata={'source': 'local'}, page_content='to understand how to make ro

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Building the RAG
rag_pdf = (
    {"context": retriever_paper | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_pdf.invoke("Tell me about the likability index")

'The likability index is a measure used to compare the likability of different robots, with results shown in Figures 8 and 9. It is used to understand how end-users perceive robots and how to build likable robots. The likability score is calculated to determine which kind of robot suits users better.'

# Let's now build a chatbot with this
You will need Gradio for this.

In [ ]:
import gradio as gr


In [ ]:
def respond(input_text, option):
    # Simple response logic (you can expand this with real chatbot logic)
    if option == 'PDF':
        response = rag_pdf.invoke(input_text)
    else:
        response = rag_web.invoke(input_text)

    return response

with gr.Blocks() as app:
    gr.Markdown("## Chat Mitra")
    with gr.Row():
        input_text = gr.Textbox(label="Your Input")
        option = gr.Radio(choices=['PDF', 'Web'], label="Choose an Option", value='PDF')
        submit_button = gr.Button("Submit")
    output_text = gr.Textbox(label="Chatbot Response")

    submit_button.click(fn=respond, inputs=[input_text, option], outputs=output_text)

app.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://59724b8a4a6821bd94.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
